In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SessionCentexETL") \
    .master("local[4]") \
    .config("spark.sql.shuffle.partitions", 4) \
    .config("spark.driver.memory", "4g") \
    .config("spark.jars", "C:\\jars\\mssql-jdbc-12.10.0.jre11.jar") \
    .getOrCreate()

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

jdbc_url = f"jdbc:sqlserver://{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')};databaseName={os.getenv('DB_NAME')};encrypt=true;trustServerCertificate=true"
properties = {
    "user": os.getenv('DB_USER'),
    "password": os.getenv('DB_PASSWORD'),
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [8]:
from pyspark.sql import functions as F

query_pa = "(select dc.ID_Agenda, ag.AgendaNombre from Doc_Compra dc inner join Agenda ag on ag.ID_Agenda = dc.ID_Agenda group by ag.AgendaNombre, dc.ID_Agenda) AS query_data"
df_nomb_emp = spark.read.jdbc(url=jdbc_url, table=query_pa, properties=properties)
repetidos = (df_nomb_emp.groupBy("AgendaNombre").count().filter(F.col("count") > 1).select("AgendaNombre"))

df_repetidos = df_nomb_emp.join(repetidos, on="AgendaNombre", how="inner")

df_repetidos.show(truncate=False)

+-------------------------------+-----------+
|AgendaNombre                   |ID_Agenda  |
+-------------------------------+-----------+
|CHAMBILLA PEREZ, YHONATAN      |10744012410|
|CHAMBILLA PEREZ, YHONATAN      |74401241   |
|AZERRAD TENSERA, WILLIAM MOISES|10004191604|
|AZERRAD TENSERA, WILLIAM MOISES|00419160   |
|ARIMUYA MORALES, JULIO         |10053435861|
|ARIMUYA MORALES, JULIO         |05343586   |
|HUAMANI CALSIN, JESSELA        |40438092   |
|HUAMANI CALSIN, JESSELA        |10404380929|
+-------------------------------+-----------+



In [ ]:
import pandas as pd
df_pd = pd.DataFrame(resultados_limpios)